# Ćwiczenia 1 - Regresja liniowa

In [ ]:
a, b = 1, 2
a+b

### Import potrzebnych bibliotek

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# 1. Wczytanie i wstępna analiza danych

In [ ]:
cars = pd.read_csv('https://raw.githubusercontent.com/knecka/plikiUP/main/CarPrices.csv')

In [ ]:
type(cars)

In [ ]:
cars.head() # .tail()

In [ ]:
cars.describe()

In [ ]:
cars.info()

### Przygotowanie danych

Wstępne założenie - interesuje nas tylko wpływ koni mechanicznych i wagi samochodu. Pozbywamy się niepotrzebnych kolumn i sprawdzamy poprawność danych w tych, które nas interesują.

In [ ]:
cars = cars[ ['horsepower', 'curbweight', 'carheight', 'price'] ]

In [ ]:
cars.info()

### Sprawdzenie, czy wszystkie wartości są wartościami liczbowymi

In [ ]:
# np.isnan(cars.values)
np.isnan?

In [ ]:
np.isnan(cars.values)

In [ ]:
np.isnan(cars.values).any()

In [ ]:
# w razie brakujacych wartosci
cars.dropna() # dla wierszy
cars.dropna(axis='columns') # dla kolumn

### Wizualizacja danych

Kilka wykresów, które na pierwszy rzut oka dadzą nam lepszą orientację w danych.

In [ ]:
plt.figure(figsize=(6,5))

plt.title('Dystrybucja cen')
sns.distplot(cars.price) # cars['price']

plt.show()

In [ ]:
plt.figure(figsize=(6,5))

plt.title('Dystrybucja cen')
plt.hist(cars.price, bins = 10)

plt.show()

In [ ]:
plt.figure(figsize=(14,5))

plt.subplot(1,2,1)
plt.title('Dystrybucja wagi')
sns.distplot(cars.curbweight)

plt.subplot(1,2,2)
plt.title('Dystrybucja KM')
sns.distplot(cars.horsepower)

plt.show()

## Oszacowanie zależności między wybranymi zmiennymi niezależnymi (cechy samochodu) a zmienną zależną (ceną)

In [ ]:
sns.pairplot(cars)

### Współczynnik korelacji między poszczególnymi atrybutami
  
[Artykuł](https://www.naukowiec.org/wiedza/statystyka/korelacja_745.html)

In [ ]:
sns.heatmap(cars.corr(), annot = True) 

plt.show()

___

# 2. Regresja liniowa

## Ogólna idea
  
[Filmik](https://www.youtube.com/watch?v=PaFPbb66DxQ)  
[Artykuł 1 Regresja liniowa](https://www.naukowiec.org/wiedza/statystyka/analiza-regresji--idea_734.html)  
[Artykuł 2 Współczynniki regresji](https://www.naukowiec.org/wiedza/statystyka/wspolczynniki-regresji-liniowej_737.html)  
[Artykuł 3 Założenia](https://www.naukowiec.org/wiedza/statystyka/zalozenia-analizy-regresji-liniowej_439.html)

## 2.1 Podział danych na treningowe i testowe

In [ ]:
X1 = cars[['horsepower']]
Y = cars[['price']] 

In [ ]:
from sklearn.model_selection import train_test_split

X1_tr, X1_test, y1_tr, y1_test = train_test_split(X1, Y, train_size = 0.8, test_size = 0.2)

## 2.2 (Prosta) Regresja liniowa

**Regresja liniowa** ogólnie - metoda oparta o liniową kombinację zmiennych niezależnych; 

**Prosta** regresja liniowa - regresja, w której używamy tylko jednej zmiennej niezależnej

In [ ]:
from sklearn.linear_model import LinearRegression

regression1 = LinearRegression()
regression1.fit(X1_tr, y1_tr)
price_prediction_horsepower = regression1.predict(X1_test)

In [ ]:
plt.figure(figsize=(10, 8))

plt.plot(X1_test, price_prediction_horsepower, color='green', label='Funkcja f(X)')
plt.scatter(X1_test, price_prediction_horsepower, color='red', label='Predykcja')
plt.scatter(X1_test, y1_test, label='Wartosc faktyczna')

plt.legend()

plt.show()

### Błąd średniokwadratowy

In [ ]:
print(y1_test.shape)
print(price_prediction_horsepower.shape)

In [ ]:
error = (price_prediction_horsepower - y1_test)
error = error.values
error

In [ ]:
np.sum(np.square(error)) / len(y1_test)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
mean_squared_error(price_prediction_horsepower, y1_test)

In [ ]:
r2_score(price_prediction_horsepower, y1_test)

# 3. Analiza wplywu innego predyktora na cenę

Powtórzenie regresji liniowej zgodnie z punktami 2.1 i 2.2, ale z użyciem innej cechy

In [ ]:
def plot_prediction(X_test, y_test, price_prediction):
    plt.figure(figsize=(10, 8))
    plt.plot(X_test, price_prediction, color='green', label='Funkcja f(X)')
    plt.scatter(X_test, price_prediction, color='red', marker='D', label='Predykcja')
    plt.scatter(X_test, y_test, label='Wartosc faktyczna')
    plt.legend()
    plt.show()

In [ ]:
plot_prediction(X_test, y_test, price_prediction)

In [ ]:
mean_squared_error(price_prediction, y_test)

# Współczynnik determinacji ($R^2$)

W uproszczeniu: miara jakości modelu, który zbudowaliśmy

https://pogotowiestatystyczne.pl/slowniczek/wspolczynnik-r-kwadrat/

https://www.naukowiec.org/wiedza/statystyka/wspolczynnik-determinacji_736.html

Miara oparta na wspolczynniku korelacji R, określa dokładnie jaka część wariancji (zmienności) wartosci wyjściowej jest wyjaśniona przez dany predyktor.

Czemu kwadrat? Latwiejszy w interpretacji: nie wiemy czy $R=0.7$ jest 1.4 razy lepszy od $R=0.5$, ale już $R^2$ jest rzeczywiście dokładnie 1.4 razy lepsze od $R^2 = 0.5$.

Jak liczymy $R^2$:
- Liczymy wariancje zmiennej zależnej (y) wokół średniej: podnosimy do kwadratu różnice między wartościami y a średnią arytmetyczną tych wartości i je sumujemy (Wariancja sredniej)
- liczymy wariancję wokol modelu (funkcji liniowej znalezionej za pomoca regresji) (Wariancja modelu)

$ R^2 = \frac{(WariancjaSredniej - WariancjaModelu)}{WariancjaSredniej} $

Jeżeli wariancja modelu jest niska (mały rozrzut wartości y wokół wyznaczonej funkcji), to wartość $R^2$ będzie bliska 1.

In [ ]:
r2_score(price_prediction, y_test)